<h1><center>Proyecto final</center></h1>


<h2><center>Cómputo conexionista (A21)</center></h2>

---

<h3><center>

José Ángel Avelar Barragan (A200361)

José de Jesús Daniel Aguirre Arzate (A200350)
</center></h3>

---

<center><h2>"Identification of patterns in cosmic-ray arrival directions using dynamic graph convolutional neural networks"</center></h2>

---

Como proyecto final nos propusimos intentar entender el articulo y tratar de replicar los resultados. A la par intentamos relacionar los conceptos vistos en clase con lo que hace el articulo. Como primer objetivo tuvimos que entender como se estan procesando los datos y como objetivo final intentar construir la red que ellos proponen (tensorflow/pytorch).

<h2><center>Implementacion</center></h2>

---

 
- <h3>Tensorflow</h3>

<h2><center>Teoria</center></h2>

---

**Problema:**
A traves de los patrones en las direcciones de arrivo de las particulas, distingir rayos cosmicos que vienen de la misma fuente o de un fondo isotropico (ruido de fondo).

Este metodo intenta encontrar patrones en las direcciones de arrivo de rayos cosmicos, si existen patrones se calsifica a las particulas como las que vienen de una fuente comun y las que vienen de direcciones de arribo isotropicas.

Datos de entrenamiento: Simulaciones simples de direcciones de arribo donde se conoce la unica fuente que genera algunos rayos cosmicos(+ detalles en el articulo acerca de la configuracion del simulador BENCHMARK1)


<p align="center">
<img width="700" height="500" src="images/skymap_0.png">
</p>


In [1]:
import numpy as np
from astrotools import skymap

import tensorflow as tf

print(f"TensorFlow version: {tf.__version__}")

TensorFlow version: 2.5.0


<h2><center>Preparacion de datos</center></h2>

---

El dataset *dataset_HAP.npz* esta organizado de la siquiente manera:

Un diccionario con las keys: data, label
En las cuales en data esta un arreglo de 50000 simulaciones de 500 particulas cada una con sus respectivas coordenadas (x,y,z, E) donde E es la energia normalizada. 

Y el arreglo que viene en label clasifica a 1 como aaquellas simulaciones que tienen una fuente o 0 como aquellas que solo son arrivos isotropicos. 

In [2]:
def train_test_split():
    file = np.load("data/cosmic_ray_sphere/dataset_HAP.npz")
    x_train, x_test = file["data"][:-10000], file["data"][-10000:]
    labels = tf.keras.utils.to_categorical(file["label"],num_classes=2)
    y_train, y_test = labels[:-10000], labels[-10000:]
    return x_train, x_test, y_train, y_test

In [3]:
X_train, X_test, y_train, y_test = train_test_split()
X_train.shape, X_test.shape

((40000, 500, 4), (10000, 500, 4))

<h4>Sample</h4>

In [4]:
test_id = 0
example_map = X_test[test_id]
skymap.eventmap(example_map[:,0:3].T, c=example_map[:,3], cblabel='Energy (normed)', opath="skymap_%i.png" % test_id)

(<Figure size 864x432 with 0 Axes>, <HammerAxes:>)

<Figure size 864x432 with 0 Axes>

In [5]:
train_points, test_points, train_features, test_features = X_train[...,:3], X_test[...,:3], X_train[...,-1, np.newaxis], X_test[...,-1, np.newaxis]
train_input_data, test_input_data = [train_points, train_features],[test_points, test_features]

#############################################

Aqui va teoria de GNN-EDgeCOnv

#############################################

In [6]:
NODES=16

LAYERS = tf.keras.layers
def kernel_nn(data, nodes=NODES):
    d1, d2 = data

    delta = LAYERS.Subtract()([d1,d2])
    x = LAYERS.Concatenate(axis=-1)([d1,delta])

    x = LAYERS.Dense(nodes, activation="relu")(x)
    x = LAYERS.BatchNormalization()(x)

    x = LAYERS.Dense(nodes, activation="relu")(x)
    x = LAYERS.BatchNormalization()(x)

    x = LAYERS.Dense(nodes, activation="relu")(x)
    x = LAYERS.BatchNormalization()(x)
    return x    

points_input = LAYERS.Input((500,3))
feats_input = LAYERS.Input((500,1))

from edgeconv import EdgeConv
x = EdgeConv(lambda a: kernel_nn(a, nodes=8), next_neighbors=5)([points_input, feats_input])
x = LAYERS.Activation("relu")(x)
x = EdgeConv(lambda a: kernel_nn(a, nodes=16), next_neighbors=8)([points_input, x])
x = LAYERS.Activation("relu")(x)
y = EdgeConv(lambda a: kernel_nn(a, nodes=32), next_neighbors=16)([points_input, x])
x = LAYERS.Activation("relu")(y)
x = LAYERS.GlobalAveragePooling1D(data_format='channels_first')(x)
out = LAYERS.Dense(2, activation="softmax")(x)

model = tf.keras.models.Model([points_input, feats_input], out)
model.summary()


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 500, 3)]     0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 500, 1)]     0                                            
__________________________________________________________________________________________________
edge_conv (EdgeConv)            (None, 500, 8)       264         input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
activation (Activation)         (None, 500, 8)       0           edge_conv[0][0]              

In [ ]:
model.compile(loss="binary_crossentropy", optimizer=tf.keras.optimizers.Adam(3E-3, decay=1E-4),metrics=['acc'])

BATCH_SIZE = 64
EPOCHS = 1
hist = model.fit(train_input_data, y_train, batch_size=BATCH_SIZE, epochs=EPOCHS)

In [ ]:
def plot_hist(history):
    fig, axes = plt.subplots(2, figsize=(12,8))
    if type(history) == dict:
        loss = history["loss"]
        acc = history["acc"]
    else:
        loss, acc = np.split(np.array(history), 2, axis=-1)
    x = np.arange(len(loss))
    axes[0].plot(x, loss, c="navy")
    axes[0].set_yscale("log")
    axes[0].set_ylabel("Loss")
    axes[1].plot(x, acc, c="firebrick")
    axes[1].set_ylabel("Accuracy")
    axes[1].set_ylim(0, 1)
    if type(history) == dict:
        axes[0].set_xlabel("Epochs")
        axes[1].set_xlabel("Epochs")
    else:
        axes[0].set_xlabel("Iterations")
        axes[1].set_xlabel("Iterations")
    fig.tight_layout()
    return fig

In [ ]:
fig = plot_hist(hist.history)
fig.savefig("./history.png")